In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install pypdf sentence-transformers qdrant-client transformers streamlit accelerate



In [5]:
PDF_FOLDER = "/content/drive/MyDrive/ncert_biology"


In [7]:
from pypdf import PdfReader

def extract_text_with_page(pdf_path):
    reader = PdfReader(pdf_path)
    pages = []
    for page_no, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            pages.append({
                "page": page_no + 1,
                "text": text
            })
    return pages


In [9]:
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]


In [16]:
!pip uninstall -y transformers sentence-transformers accelerate
!pip install transformers==4.38.2 sentence-transformers==2.6.1 accelerate==0.27.2


Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: sentence-transformers 5.2.2
Uninstalling sentence-transformers-5.2.2:
  Successfully uninstalled sentence-transformers-5.2.2
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 115.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2


In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient(":memory:")

client.recreate_collection(
    collection_name="ncert_biology",
    vectors_config=VectorParams(
        size=384,
        distance=Distance.COSINE
    )
)


/tmp/ipython-input-742252397.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [10]:
import os
from qdrant_client.models import PointStruct # Added import

points = []
idx = 0

for file in os.listdir(PDF_FOLDER):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(PDF_FOLDER, file)
        pages = extract_text_with_page(pdf_path)

        for p in pages:
            chunks = chunk_text(p["text"])
            for chunk in chunks:
                vector = embedding_model.encode(chunk).tolist()

                points.append(
                    PointStruct(
                        id=idx, # Used PointStruct to define the point
                        vector=vector,
                        payload={
                            "pdf": file,
                            "page": p["page"],
                            "text": chunk
                        }
                    )
                )
                idx += 1

client.upsert(collection_name="ncert_biology", points=points)

print("✅ PDFs indexed into Qdrant")

✅ PDFs indexed into Qdrant


In [11]:
def retrieve_context(query, top_k=5):
    query_vector = embedding_model.encode(query).tolist()

    hits = client.search_points(
        collection_name="ncert_biology",
        vector=query_vector,
        limit=top_k
    )

    context = ""
    sources = []

    for hit in hits:
        context += hit.payload["text"] + "\n\n"
        sources.append(f"{hit.payload['pdf']} | Page {hit.payload['page']}")

    return context, sources


In [ ]:
def retrieve_context(query, top_k=5):
    query_vector = embedding_model.encode(query).tolist()

    hits = client.search(
        collection_name="ncert_biology",
        query_vector=query_vector,
        limit=top_k
    )

    context = ""
    sources = []

    for hit in hits:
        context += hit.payload["text"] + "\n\n"
        sources.append(f"{hit.payload['pdf']} | Page {hit.payload['page']}")

    return context, sources


In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
def generate_answer(question):
    if not question.strip():
        return "Please ask a question.", []

    context, sources = retrieve_context(question)

    if not context.strip():
        return "No relevant content found in PDFs.", []

    prompt = (
        "Answer the question using the context below.\n\n"
        f"Context:\n{context[:1500]}\n\n"
        f"Question:\n{question}\n\n"
        "Answer:"
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer, sources


In [14]:
!pip install pyngrok


In [17]:
import gradio as gr

def ask(question):
    answer, sources = generate_answer(question)
    return answer, "\n".join(set(sources))

iface = gr.Interface(
    fn=ask,
    inputs="text",
    outputs=["text", "text"],
    title="📘 NCERT Biology – AI PDF Search (RAG)"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f35111d8b48cb7c16.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
